# Import GitHub repository

In [1]:
import os
import logging
import sys

In [2]:
if not os.path.isdir('./owr'):
  !git clone -b icarl https://github.com/gianluigilopardo/Open-World-Recognition.git
  !mv 'Open-World-Recognition' 'owr'

# Import packages

In [3]:
from owr import ResNet
from owr.dataset import Subset
from owr.icarl import classify
from owr.icarl import incremental_train
from owr.icarl import update_representation
from owr.icarl import construct_exemplar_set
from owr.icarl import reduce_exemplars
from owr.icarl import generate_new_exemplars
from owr.models import compute_loss
from owr.models import train_network
from owr import params
from owr.utils import get_classes_names
from owr.utils import get_task_indexes
from owr.utils import splitter
from owr.utils import map_splits
from owr.utils import get_classes
from owr.utils import get_indexes
import owr.models

In [4]:
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn import functional as F
import random

In [5]:
print(params.SEED)
print(params.NUM_WORKERS)

42
2


# Define Datasets

In [6]:
train_transformer = transforms.Compose([transforms.RandomCrop(size = 32, padding=4),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transformer = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [7]:
from torchvision import datasets
trainDS = datasets.cifar.CIFAR100( 'data', train=True, download=True, transform=train_transformer)
testDS = datasets.cifar.CIFAR100( 'data', train=False, download=True, transform=test_transformer)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
splits = splitter()

# Define Network

In [9]:
ICaRL = ResNet.resnet32(num_classes=100)
ICaRL =  ICaRL.to(params.DEVICE)

In [10]:
exemplars = [None]*params.NUM_CLASSES

test_indexes =  []
accs = []
classifier = 'nme'

In [11]:
from sklearn.neighbors import KNeighborsClassifier

In [12]:
def print_stats(preds, true, col):
  preds_old_classes = 0
  preds_new_classes = 0
  true_new_classes = 0
  true_old_classes = 0

  for cl in col[:-10]:
    mapped_class = map_splits([cl], col)
    for predicted in preds:
      if mapped_class == predicted:
        preds_old_classes += 1
    for actual in true:
      if mapped_class == actual:
        true_old_classes += 1

  for cl in col[-10:]:
    mapped_class = map_splits([cl], col)
    for predicted in preds:
      if mapped_class == predicted:
        preds_new_classes += 1
    for actual in true:
      if mapped_class == actual:
        true_new_classes += 1

  print('Tot predictions new classes:', preds_new_classes, 'over a total of', true_new_classes)
  print('Tot predictions old classes:', preds_old_classes, 'over a total of', true_old_classes )
  return

In [ ]:
for task in range(0, params.NUM_TASKS*params.TASK_SIZE, params.TASK_SIZE):
    train_indexes = get_task_indexes(trainDS, task)
    test_indexes = test_indexes + get_task_indexes(testDS, task)

    train_dataset = Subset(trainDS, train_indexes, transform = train_transformer)
    test_dataset = Subset(testDS, test_indexes, transform = test_transformer)

    train_loader = DataLoader( train_dataset, num_workers=2, batch_size=params.BATCH_SIZE, shuffle=True)
    test_loader = DataLoader( test_dataset, num_workers=2, batch_size=params.BATCH_SIZE , shuffle=True )
    print(task)
    ICaRL, exemplars = incremental_train(trainDS, ICaRL, exemplars, task, train_transformer)
    col = []
    for i,x in enumerate( splits[ :int(task/10) + 1]) : 
        v = np.array(x)
        col = np.concatenate( (col,v), axis = None)
        col = col.astype(int)
    mean = None
    total = 0.0
    running_corrects = 0.0
    
    for img, lbl, _ in train_loader:
        img = img.float().to(params.DEVICE)
        preds, mean = classify(img, exemplars, ICaRL, task, trainDS, mean)
        preds = preds.to(params.DEVICE)
        labels = map_splits(lbl, col).to(params.DEVICE)
        total += len(lbl)
        running_corrects += torch.sum(preds == labels.data).data.item()

    accuracy = float(running_corrects/total)
    print(f'task: {task}', f'train accuracy = {accuracy}')

    total = 0.0
    running_corrects = 0.0
    tot_preds = []
    tot_lab = []
    for img, lbl, _ in test_loader:
        img = img.float().to(params.DEVICE)
        preds, _ = classify(img, exemplars, ICaRL, task, trainDS, mean)
        preds = preds.to(params.DEVICE)
        labels = map_splits(lbl, col).to(params.DEVICE)
        tot_preds = np.concatenate( ( tot_preds, preds.data.cpu().numpy() ) )
        tot_lab = np.concatenate( (tot_lab, labels.data.cpu().numpy()  ) )
        total += len(lbl)
        running_corrects += torch.sum(preds == labels.data).data.item()
    print_stats(tot_preds, tot_lab, col)

    accuracy = float(running_corrects/total)
    print(f'task: {task}', f'test accuracy = {accuracy}')